In [1]:
# Imports
import os
import json
import tiktoken
from openai import AzureOpenAI
from dotenv import load_dotenv

# Environment setup
load_dotenv()

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
key = os.getenv("AZURE_SEARCH_KEY") 
verbose = False #Set to true to see more output information
DATA_DIR = "/data/extracted"

#Initialize AzureOpenAI client
client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],  
  api_version = "2023-12-01-preview"
  )

messages=[]

# count tokens
def count_tokens(prompt) -> int:  
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    token_sizes = len(encoding.encode(prompt))
    return token_sizes

#lookup functions
rag_lib="" 

def get_library_info(lib_name: str)->str:  
    if lib_name=="CAA File":    
        return ""
    else:
        return rag_lib
def get_function_info(lib_name: str, fn_name:str)->str: 
    if fn_name=="FB_SendEMail" or fn_name=="FB_Pop3EMailClient" or fn_name=="FB_Print":  
        with open(f'../{DATA_DIR}/{fn_name}.txt', 'r') as file:
            response=file.read()
            return response    
    else:
        return ""

    
available_functions = {
                "get_library_info": get_library_info,
                "get_function_info": get_function_info                
        }

#GPT tools
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_library_info",
                "description": "Get the information about the functions included in a library.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "lib_name": {
                            "type": "string",
                            "description": "The name of the library to get information about. (examples: FileFormatUtility,HttpHandling,TimeSync)",
                        },
                        "unit": {"type": "string"},
                    },
                    "required": ["lib_name"],
                },
            },
        },
           {
            "type": "function",
            "function": {
                "name": "get_function_info",
                "description": "Use this function to know how to use any function whose name starts with FB_ (example FB_CalcVector,FB_Rotfl).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "lib_name": {
                            "type": "string",
                            "description": "The name of the library that contains the function. (examples: FileFormatUtility,HttpHandling,TimeSync)",
                        },
                        "unit": {"type": "string"},
                        "fn_name":{
                            "type":"string",               
                            "description": "The name of the function to find (examples: FB_CalcVector,FB_CutMaterial)"
        }, 
                    },
                    "required": ["lib_name", "fb_name"],
                },
            },
        },        
    ]

def load_external_info(response_message):
    tool_calls = response_message.tool_calls    
    if tool_calls:        
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print(f"\nModel wants to call a function '{function_name}' with arguments: {function_args}") 
            function_response = function_to_call(**function_args)
            
            total_tokens=count_tokens(function_response)
            print (f'Total function tokens: {total_tokens}')
            
            messages.append(response_message) 
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  

def generate_code():    
    openai_response = client.chat.completions.create(
        model=deployment,    
        messages = messages,
        temperature=0.3,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        tools=tools
    )
    return openai_response.choices[0].message

In [2]:
# user_command = f'''
# Generate a small program in IEC61131-3 that sends an email using following parameters:
# 1. To: "receiver@se.com"
# 2. Subject: "Test email"
# 3. Body: "This is a test email"
# 4. From: "sender@se.com"

# Authentication required.
# Message should be sent with high priority.
# Use Login to authenticate using "corrado" as user and "p@ssw0rd123" as password.
# Verify that the email has been sent successfully, if not print the error message.
# User protocol TLS 1.3

# '''

In [3]:
# user_command = f'''
# Generate a small program in IEC61131-3 that receives an email using following parameters:
# 1. from: "bart_simpson@foo.com"
# 2. if received email contains "joke" print an email with following parameters:
#     1. To: "alert@foo.com"
#     2. Subject: "Alert email"
#     3. Body: "A joke email has been received"
#     4. From: "sender@se.com"
#     5. Authentication required.
#     6. Use Protocol TLS 1.3
#     7. Message should be sent with low priority.
#     8. No Authentication required.
#     9. Verify that the email has been sent successfully, if not wait 5 seconds and retry up to 3 times before aborting the whole operation.
#     10. Consider all the case of failure, for example if the email server is not reachable.

#  '''
 
user_command = f'''
Generate a small program in IEC61131-3 that receives an email using following parameters:
1. from: "bart_simpson@foo.com"
2. if received email contains "joke",
  - blow the email to console.
 '''
 

In [4]:

library_rules = '''
These are the rules to follow to retrieve the information about an external library:
1. if name of the library is with "CAA File" don't invoke 'get_library_info' nor 'get_function_info'.
2. 'get_library_info' return information about the functions included in a library, it must to be the first function to call.
3. call 'get_function_info' to retrieve the information about the function you need to use.

example:
1. You need to use one or more function contained into 'Lib001' library.
2. You need to call get_library_info with lib_name = 'Lib001'.
3. The function return info about the library stating it contains 2 functions: 'FB_RxData' and 'FB_TxData'.
4. if you need to use 'FB_RxData' you need to call get_function_info with lib_name = 'Lib0001 and fn_name = 'FB_RxData'.
5. if you need to use 'FB_TxData' you need to call get_function_info with lib_name = 'Lib001' and fn_name = 'FB_TxData'.
'''

In [5]:
# user_command = f'''
# Generate a small program in IEC61131-3 that receives all the emails sent to the following email address: "bart_simpson@foo.com" then for each email received, do the following:
# 2. if received email contains "spam" send an email with following parameters:
#     1. To: "spam@foo.com"
#     2. Subject: "Alert spam email"
#     3. Body: "A spam email has been received"
#     4. From: "guardian@se.com"
#     5. No Authentication required.
#     6. Message should be sent with low priority.    
#     7. Verify that the email has been sent successfully, if not wait 5 seconds and retry up to 3 times before aborting the whole operation.    
# 3. else saves the content of the email to a file named "print_email.txt".


#'''

In [6]:
rag_lib='''
1. `FB_SendEMail`: This function block includes the related functions for sending emails. Each instance handles one SMTP connection. It allows you to send emails. After the function block has been enabled and is being executed, a TCP connection to the email server is established using the user credentials that have been submitted using iq_stCredentials.

2. `FB_Pop3EMailClient`: This function block includes the related functions for receiving and deleting emails using POP3. Each instance handles one POP3 connection. It allows you to receive and delete emails. By using attachments of received emails you are able to get input for several system features which are based on files located on the system memory.

3. `FC_EtResultToString`: This function is used to convert an enumeration element of type ET_Result to a variable of type STRING. It provides a way to convert the enumeration type to a string for easier interpretation and display.

4. `FB_BlowTxt`: This function is used to blow some text content to a generic output: console, file, memory.
'''

In [7]:
libraries='''
Library title,Company,Category,Component,Description
CollisionDetection,Schneider Electric,Application,PacDrive Robotics,This library offers a set of functions to perform a collision check between two or more objects and a distance calculation between two or more objects. For more information, refer to the CollisionDetection Library Guide.
EMailHandling,Schneider Electric,Communication,Internet Protocol Suite, This library supports the implementation of an email client and provides function blocks for sending SMTP (Simple Mail Transfer Protocol) and receiving POP3 (Post Office Protocol) emails.NOTE: The communication is implemented using the TcpUdpCommunication library.AlarmManager,intern,Intern > AlarmManager,Core Repository,This library contains the components for the alarm handling.
CAA File,CAA Technical Workgroup,System,Core Repository,This library provides function blocks for accessing file directory systems and files.

'''
library_system_prompt=f'''
You are an assistant able to select a library based on input requirements.
Your goal is to return the library names selecting from the one provided in the context.
Favor the use of external libraries over internal libraries.
Return only the name of the libraries that matches the user input separated by a comma, if name of the library starts with CAA then append "from CodeSys" to the name of the library.

CONTEXT:
```
{libraries}
```
'''

In [8]:

# Detect libraries to use
messages=[]
messages.append({'role': 'system', 'content': library_system_prompt})
messages.append({'role': 'user', 'content': user_command})   

openai_response = client.chat.completions.create(
        model=deployment,    
        messages = messages,
        temperature=0.3,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)

target_libraries= openai_response.choices[0].message.content
print(target_libraries)

Based on your requirements, you need a library that supports the implementation of an email client and provides function blocks for receiving emails. From the provided context, the `EMailHandling` library from Schneider Electric fits this requirement. However, since you are asking for a program in IEC 61131-3 standard, please note that I can only provide a pseudo-code representation as actual implementation may vary depending on the specific PLC programming environment and available function blocks.

Here is a pseudo-code example using the `EMailHandling` library:

```iec
PROGRAM ReceiveEmailJoke
VAR
    emailClient : EMailClient; (* Instance of the email client *)
    email : Email; (* Variable to store the received email *)
    result : BOOL; (* Result of the email receive operation *)
    jokeFound : BOOL; (* Indicator if "joke" is found in the email *)
END_VAR

(* Initialize the email client with the necessary parameters *)
emailClient.Server := 'pop3.foo.com'; (* Replace with actu

In [9]:
# system message
system_message = f'''
You are an assistant with knowledge of the following topics:
1. IEC61131-3 languages
2. Structured Text
3. Function Block Diagram
4. IEC61131-3 coding standards
5. IEC61131-3 best practices
6. IEC61131-3 coding guidelines
7. IEC61131-3 programming
8. IEC61131-3 programming languages
9. Schneider Electric EcoStruxure Control Expert
10. Schneider Electric EcoStruxure Machine Expert
11. Schneider Electric EcoStruxure Machine Expert Libraries and Templates

Your job is to generate small examples of code using exclusively IEC61131-3 Structured Text base on user input.
You can assume that all the code will be executed on a Schneider Electric EcoStruxure Control Expert or Schneider Electric EcoStruxure Machine Expert PLC and that all libraries are available.

The code should make use of the following {target_libraries} libraries.

In order to understand how to use the libraries, you can use the following indications:

```
{library_rules}
```

'''

In [10]:
messages=[]
messages.append({'role': 'system', 'content': system_message})
messages.append({'role': 'user', 'content': user_command})   

sys_token=count_tokens(system_message)
cmd_tokens=count_tokens(user_command)

print(f"system_message tokens: {sys_token}")
print(f"user_message tokens: {cmd_tokens}")

isCodeGenerated=False

while(isCodeGenerated==False): 
    print("Generating code...")    
    response = generate_code()    
    if (response.tool_calls):
        load_external_info(response)
    else:
        isCodeGenerated=True
        print("Code generation completed!")        
        print(response.content)

system_message tokens: 913
user_message tokens: 52
Generating code...

Model wants to call a function 'get_library_info' with arguments: {'lib_name': 'EMailHandling'}
Total function tokens: 214
Generating code...

Model wants to call a function 'get_function_info' with arguments: {'lib_name': 'EMailHandling', 'fn_name': 'FB_Pop3EMailClient'}
Total function tokens: 3898
Generating code...
Code generation completed!
Based on the information provided about the `FB_Pop3EMailClient` function block from the `EMailHandling` library, here is an example of a Structured Text program that receives an email and checks if it contains the word "joke". If it does, the program will blow the email to the console using the `FB_BlowTxt` function block.

```iec
PROGRAM ReceiveEmailAndPrintJoke
VAR
    pop3Client : FB_Pop3EMailClient; // Instance of the POP3 client
    credentials : ST_CredentialsReceiveEMail; // Credentials for the email server
    inbox : ARRAY [1..10] OF ST_Email; // Array to store rece